# 3. Spark Structured Streaming을 통해 원하는 단어만 받기

SparkSession을 생성하고 소켓으로 데이터를 수신하도록 스트리밍 설정을 하고 원하는 형태의 데이터만 다룰 수 있도록 합니다.

## 기본 스트리밍 설정

### TODO: 주석과 참고 페이지를 참고하여 실시간 스트리밍 데이터 처리를 가능하도록 완성해주세요.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, current_timestamp, window, expr

# 기존 Spark 세션 종료 후 새로운 세션 생성
# 현재 세션이 존재하는 경우 종료하고, 새로운 세션을 시작합니다.
spark = SparkSession.builder.appName("WordCountWithWatermarking").getOrCreate()

In [1]:
# 소켓을 통한 스트리밍 데이터 수신 설정
# localhost의 9999 포트에서 텍스트 데이터를 실시간으로 수신하여 스트림을 생성합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.format.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.readStream.html
lines = spark._____________._____________("socket").option("host", "localhost").option("port", 9999).load()

In [ ]:
# 현재 시간 열 추가 및 단어 분리
# - 수신된 데이터에 현재 타임스탬프 열(timestamp)을 추가하여 이벤트 기반 시간 처리가 가능하게 합니다.
# - explode와 split 함수를 사용해 각 줄(line)을 공백(" ")으로 나누고 개별 단어를 행(row)으로 만듭니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.current_timestamp.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.explode.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.split.html
lines_with_timestamp = lines.withColumn("timestamp", _____________())
words = lines_with_timestamp.select(_____________(split(lines.value, " ")).alias("word"), "timestamp")

In [ ]:

# 특정 단어 필터링
# - 특정 단어 ('error', 'fail', 'warning')가 포함된 데이터만 필터링하여 집계에 포함합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.expr.html
alerts = words.filter(_____________("word IN ('error', 'fail', 'warning')"))

In [ ]:
# 워터마크 및 윈도우를 적용하여 단어 빈도 집계
# - 워터마크 설정 (10분): 10분이 지난 오래된 데이터는 집계에서 제외합니다.
# - 윈도우 설정 (5초 간격): 단어의 빈도를 5초 고정 윈도우로 집계하고, 5초마다 슬라이딩하여 갱신합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withWatermark.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.window.html
alertCounts = alerts._____________("timestamp", "10 minutes") \
    .groupBy("word", window("timestamp", "5 seconds", "5 seconds")).count()

### Output Mode 설정
* Complete: 모든 집계 결과를 출력
* Append: 추가된 데이터만 출력
* Update: 변경된 데이터만 갱신하여 출력


In [ ]:
# 실시간 출력 설정
# - outputMode("update"): 변경된 데이터만 출력하여 갱신합니다.
# - format("console"): 콘솔에 결과를 출력합니다.
# - option("truncate", "false"): 출력 시 내용을 잘리지 않도록 설정합니다.
# - trigger(processingTime="5 seconds"): 5초마다 갱신된 데이터를 수집 및 출력합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.streaming.DataStreamWriter.outputMode.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.writeStream.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.streaming.DataStreamWriter.trigger.html
query = alertCounts._____________ \
    ._____________("update") \
    ._____________("console") \
    .option("truncate", "false") \
    .trigger(processingTime="5 seconds") \
    .start()

# 스트리밍 쿼리를 계속 실행하여 데이터 수신을 대기합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.streaming.StreamingQuery.awaitTermination.html
query.awaitTermination()

In [ ]:
spark.stop()